In [ ]:
import pandas as pd

disease_df = pd.read_csv("/path_to_file/disease_file.csv")  # 상병명 파일
claim_df = pd.read_csv("/path_to_file/claim_file.csv")  # 보험 청구 항목 파일

merged_df = pd.merge(disease_df, claim_df, how="left", on="상병코드")

print(merged_df.head())

merged_df["보험 청구 항목"].fillna("항목 없음", inplace=True)

merged_df.to_csv("/path_to_output/merged_disease_claims.csv", index=False)

In [ ]:
import pandas as pd

# 파일에서 병명과 코드가 있는 데이터 읽어오기
disease_df = pd.read_csv("/path_to_file/disease_file.csv")  # 병명 및 상병 코드 데이터
insurance_df = pd.read_csv("/path_to_file/insurance_file.csv")  # 보험 청구 항목 데이터

# 두 데이터프레임 병합 (상병 코드를 기준으로)
merged_df = pd.merge(disease_df, insurance_df, how="left", on="상병코드")

# 결과 출력
print(merged_df.head())

# 결과 저장
merged_df.to_csv("/path_to_output/merged_disease_insurance.csv", index=False)

In [ ]:
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# 인증 설정
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SERVICE_ACCOUNT_FILE = "credentials.json"  # 서비스 계정 파일 경로

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# 구글 시트 API 빌드
service = build("sheets", "v4", credentials=creds)

# 시트 ID 및 범위 정의
SPREADSHEET_ID = "your_spreadsheet_id"
insurance_claims_range = "Sheet1!A:E"  # 보험 청구 항목이 있는 시트 범위
disease_names_range = "Sheet2!A:B"  # 질병명 코드가 있는 시트 범위

# 보험 청구 항목 데이터 불러오기
result = (
    service.spreadsheets()
    .values()
    .get(spreadsheetId=SPREADSHEET_ID, range=insurance_claims_range)
    .execute()
)
insurance_claims_data = result.get("values", [])

# 질병명 코드 데이터 불러오기
result = (
    service.spreadsheets()
    .values()
    .get(spreadsheetId=SPREADSHEET_ID, range=disease_names_range)
    .execute()
)
disease_names_data = result.get("values", [])


# 보험 청구 항목을 병명 코드와 매칭하는 함수
def map_claims_to_diseases(disease_data, claim_data):
    for disease in disease_data[1:]:  # 헤더 제외
        disease_code = disease[0]
        for claim in claim_data[1:]:  # 헤더 제외
            if disease_code in claim[3]:  # 코드 열 확인
                disease.append(claim[1])  # 세부 청구 항목 추가
            else:
                disease.append("")  # 매칭되지 않으면 빈 값 추가
    return disease_data


# 데이터를 매핑
updated_disease_data = map_claims_to_diseases(disease_names_data, insurance_claims_data)

# 업데이트된 데이터를 구글 시트로 다시 쓰기
service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=disease_names_range,
    valueInputOption="RAW",
    body={"values": updated_disease_data},
).execute()

In [4]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

# 데이터 추출
insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 코드 매칭 및 새 열 추가
for disease_row_idx, disease_row in enumerate(
    disease_code_data[1:], start=2
):  # 헤더 제외, 행 번호 추적
    disease_code = disease_row[0]  # 상병 코드는 첫 번째 열에 있다고 가정
    category = None
    item = None
    for claim_row in insurance_claim_data[1:]:
        claim_codes = claim_row[3]  # 코드 열은 네 번째 열에 있다고 가정
        if any(code.strip() == disease_code for code in claim_codes.split(",")):
            # 보험 청구 Category와 세부 항목을 저장
            category = claim_row[0]  # 보험 청구 Category
            item = claim_row[1]  # 세부 청구 항목
            break  # 매칭된 항목을 찾으면 반복 종료

    # 매칭된 데이터가 있으면 D, E 열에 업데이트
    if category and item:
        disease_code_sheet.update(
            f"D{disease_row_idx}:E{disease_row_idx}", [[category, item]]
        )

        # 요청 사이에 2초의 지연을 추가하여 API 속도 제한에 맞추기
        time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_22768\2077840672.py:40: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update(f'D{disease_row_idx}:E{disease_row_idx}', [[category, item]])


KeyboardInterrupt: 

In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

# 데이터 추출
insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 코드 매칭 및 새 열 추가
for disease_row_idx, disease_row in enumerate(
    disease_code_data[1:], start=2
):  # 헤더 제외, 행 번호 추적
    disease_code = disease_row[0]  # 상병 코드는 첫 번째 열에 있다고 가정
    category = None
    item = None
    for claim_row in insurance_claim_data[1:]:
        claim_codes = claim_row[3].split(
            ","
        )  # 코드 열은 네 번째 열에 있다고 가정, 코드를 쉼표로 분리

        # 코드 리스트에서 정확히 일치하는 코드가 있는지 확인
        if disease_code in [code.strip() for code in claim_codes]:
            # 보험 청구 Category와 세부 항목을 저장
            category = claim_row[0]  # 보험 청구 Category
            item = claim_row[1]  # 세부 청구 항목
            break  # 매칭된 항목을 찾으면 반복 종료

    # 매칭된 데이터가 있으면 D, E 열에 업데이트
    if category and item:
        disease_code_sheet.update(
            f"C{disease_row_idx}:D{disease_row_idx}", [[category, item]]
        )
        time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_22768\3306577767.py:40: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update(f'D{disease_row_idx}:E{disease_row_idx}', [[category, item]])


In [9]:
import gspread
import re  # 정규 표현식을 위한 모듈
import time
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

# 데이터 추출
insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()


for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    if "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(
                ","
            )  # 코드 열은 네 번째 열에 있다고 가정, 코드를 쉼표로 분리

            if disease_code in [code.strip() for code in claim_codes]:

                category = claim_row[0]
                item = claim_row[1]
                break

        clean_category = re.sub(r"[0-9.]", "", category).strip()
        clean_item = re.sub(r"[0-9.]", "", item).strip()

    disease_code_sheet.update(
        f"C{disease_row_idx}:D{disease_row_idx}", [[clean_category, clean_item]]
    )
    time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_22768\3366742684.py:58: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update(


In [12]:
import gspread
import re
import time
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

doc = gc.open_by_url(spreadsheet_url)

insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 추가된 열의 이름 저장
disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])

# 상병 코드를 추적하여 중복될 경우 항목을 병합할 딕셔너리 생성
disease_code_tracker = {}

for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    if not disease_code.strip() or "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(",")

            if disease_code in [code.strip() for code in claim_codes]:
                new_category = re.sub(r"[0-9.]", "", claim_row[0]).strip()
                new_item = re.sub(r"[0-9.]", "", claim_row[1]).strip()

                # 중복된 상병 코드를 처리하여 병합
                if disease_code in disease_code_tracker:
                    clean_category, clean_item = disease_code_tracker[disease_code]

                    # 중복된 항목 병합
                    if new_category not in clean_category:
                        clean_category += f", {new_category}"
                    if new_item not in clean_item:
                        clean_item += f", {new_item}"
                else:
                    clean_category = new_category
                    clean_item = new_item

                # 병합된 결과를 딕셔너리에 저장
                disease_code_tracker[disease_code] = (clean_category, clean_item)

        # 병합된 데이터를 시트에 업데이트
        if disease_code in disease_code_tracker:
            final_category, final_item = disease_code_tracker[disease_code]
            disease_code_sheet.update(
                f"C{disease_row_idx}:D{disease_row_idx}", [[final_category, final_item]]
            )
            time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_5588\294322670.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])
C:\Users\user\AppData\Local\Temp\ipykernel_5588\294322670.py:65: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update(


KeyboardInterrupt: 

In [1]:
import gspread
import re
import time
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

doc = gc.open_by_url(spreadsheet_url)

insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 추가된 열의 이름 저장
disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])

# 상병 코드를 추적하여 중복될 경우 항목을 병합할 딕셔너리 생성
disease_code_tracker = {}

for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    # 상병 코드에 '~'가 포함된 경우 빈칸으로 처리
    if not disease_code.strip() or "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"
        matched = False  # 매칭 여부를 추적하는 변수

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(",")

            if disease_code in [code.strip() for code in claim_codes]:
                new_category = re.sub(r"[0-9.]", "", claim_row[0]).strip()
                new_item = re.sub(r"[0-9.]", "", claim_row[1]).strip()

                matched = True  # 매칭이 발생했음을 표시

                # 중복된 상병 코드를 처리하여 병합
                if disease_code in disease_code_tracker:
                    clean_category, clean_item = disease_code_tracker[disease_code]

                    # 중복된 항목 병합
                    if new_category not in clean_category:
                        clean_category += f", {new_category}"
                    if new_item not in clean_item:
                        clean_item += f", {new_item}"
                else:
                    clean_category = new_category
                    clean_item = new_item

                # 병합된 결과를 딕셔너리에 저장
                disease_code_tracker[disease_code] = (clean_category, clean_item)

        # 매칭된 코드가 없으면 '일치하는 보험 상품이 없습니다' 추가
        if not matched:
            clean_category = "일치하는 보험 상품이 없습니다"
            clean_item = "일치하는 보험 상품이 없습니다"

    # 병합된 데이터를 시트에 업데이트
    disease_code_sheet.update(
        f"C{disease_row_idx}:D{disease_row_idx}", [[clean_category, clean_item]]
    )
    time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_15944\1815356176.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])
C:\Users\user\AppData\Local\Temp\ipykernel_15944\1815356176.py:72: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  disease_code_sheet.update(


In [6]:
!pip install pandas transformers fuzzywuzzy


In [22]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 특정 시트 선택 (사본 페이지)
sheet = doc.worksheet("상병 Category의 사본")

# 시트의 모든 데이터를 가져오기 (2D 리스트 형태로 가져옴)
data = sheet.get_all_values()

# 데이터프레임으로 변환
df = pd.DataFrame(data[1:], columns=data[0])  # 첫 번째 행은 컬럼명으로 설정


# 검색 함수
def search_in_dataframe(query, dataframe):
    # 입력된 query에 해당하는 모든 열에서 검색
    result = dataframe[
        dataframe.apply(
            lambda row: row.astype(str).str.contains(query, case=False, na=False).any(),
            axis=1,
        )
    ]
    return result


# 사용자 입력 예시
user_input = "외과"  # 사용자가 입력한 검색어

# 데이터프레임 내에서 검색 수행
search_results = search_in_dataframe(user_input, df)

# 결과 출력
print("Search Results:")
print(search_results)

Search Results:
    불완전 상병 코드                            불완전 상병명  \
5      K00.10                            전치 부위 과잉치   
6      K00.11                           소구치 부위 과잉치   
7      K00.12                           대구치 부위 과잉치   
8      K00.19                            상세불명의 과잉치   
15     K00.44                           절렬 (치아 만곡)   
20     K00.63                     잔존 [지속성][탈락성] 유치   
21     K00.68                      치아맹출의 기타 명시된 장애   
22     K00.69                        치아맹출의 상세불명 장애   
24      K01.0                                  매몰치   
34     K01.18                               과잉 매복치   
35     K01.19                            상세불명의 매복치   
38      K02.1                               상아질 우식   
39      K02.2                              시멘트질 우식   
42      K02.8                              기타 치아우식   
49      K03.5                              치아의 강직증   
60      K04.5                             만성 근단치주염   
61     K04.62           구강으로 연결된 동 (동이 있는 근단주위 농양)   
62      K04.7 

In [2]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from unicode import join_jamos
from rapidfuzz import process, fuzz

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 특정 시트 선택 (사본 페이지)
sheet = doc.worksheet("상병 Category의 사본")

# 시트의 모든 데이터를 가져오기 (2D 리스트 형태로 가져옴)
data = sheet.get_all_values()

# 데이터프레임으로 변환
df = pd.DataFrame(data[1:], columns=data[0])  # 첫 번째 행은 컬럼명으로 설정


class HangulSearch:
    eng_to_kor_dict = {
        "r": "ㄱ",
        "R": "ㄲ",
        "s": "ㄴ",
        "e": "ㄷ",
        "E": "ㄸ",
        "f": "ㄹ",
        "a": "ㅁ",
        "q": "ㅂ",
        "Q": "ㅃ",
        "t": "ㅅ",
        "T": "ㅆ",
        "d": "ㅇ",
        "w": "ㅈ",
        "W": "ㅉ",
        "c": "ㅊ",
        "z": "ㅋ",
        "x": "ㅌ",
        "v": "ㅍ",
        "g": "ㅎ",
        "k": "ㅏ",
        "o": "ㅐ",
        "i": "ㅑ",
        "O": "ㅒ",
        "j": "ㅓ",
        "p": "ㅔ",
        "u": "ㅕ",
        "P": "ㅖ",
        "h": "ㅗ",
        "y": "ㅛ",
        "n": "ㅜ",
        "b": "ㅠ",
        "m": "ㅡ",
        "l": "ㅣ",
    }

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def convert_eng_to_kor(self, eng_text):
        """영어 입력을 한글 자모로 변환하는 함수"""
        return "".join(self.eng_to_kor_dict.get(char, char) for char in eng_text)

    def correct_query(self, query):
        """검색어를 DataFrame의 데이터를 참고하여 보정하는 함수"""
        if all(
            char.isalpha() and char.islower() for char in query
        ):  # 모든 문자가 알파벳 소문자인 경우 한글로 변환
            query = join_jamos(self.convert_eng_to_kor(query))

        max_score = 0
        corrected = query
        for column in self.dataframe.columns:
            closest_match = process.extractOne(
                query, self.dataframe[column], scorer=fuzz.WRatio
            )
            if closest_match and closest_match[1] > max_score:
                max_score = closest_match[1]
                corrected = (
                    closest_match[0] if max_score > 50 else query
                )  # 유사도 50% 이상인 경우만 반환

        return corrected

    def search_in_dataframe(self, query):
        # 입력된 query에 해당하는 모든 열에서 부분일치 검색
        result = self.dataframe[
            self.dataframe.apply(
                lambda row: row.astype(str)
                .str.contains(query, case=False, na=False)
                .any(),
                axis=1,
            )
        ]

        # 부분일치 결과가 없으면 오탈자 보정을 통해 검색
        if result.empty:
            corrected_query = self.correct_query(query)
            result = self.dataframe[
                self.dataframe.apply(
                    lambda row: row.astype(str)
                    .str.contains(corrected_query, case=False, na=False)
                    .any(),
                    axis=1,
                )
            ]
            return result, corrected_query

        return result, None


# 사용자 입력 예시
user_input = "몰치"  # 사용자가 입력한 검색어

# HangulSearch 클래스 인스턴스 생성 및 검색 수행
searcher = HangulSearch(df)
search_results, corrected_query = searcher.search_in_dataframe(user_input)

# 결과 출력
if corrected_query:
    print(
        f"'{user_input}'에 대한 검색 결과가 없습니다. 대신 '{corrected_query}'로 검색한 결과입니다:"
    )
else:
    print(f"'{user_input}'에 대한 검색 결과입니다:")

search_results

'몰치'에 대한 검색 결과입니다:


,불완전 상병 코드,불완전 상병명,청구 카테고리,세부 청구 항목
23,K01.~,매몰치 및 매복치,,
24,K01.0,매몰치,외과,"단순매복, 복잡매복, 완전매복"
112,K07.35,위치이상을 동반한 매몰치 또는 매복치,"청구 기초 및 기본진료, 외과","CT (치과용), 복잡매복"
